# Adatelőkészítés a pandas csomag segítségével

Az adatelőkészítés fázisa számos különféle műveletet foglal magában. Jelen workbook-ban olyan mintapéldákat tekintünk át, amelyek ezen lehetséges műveletekből igyekeznek minél többet bemutatni, de természetesen nem fedik a teljes témakört.  

## Dummy változó létrehozása

A dummyfikálás bemutatásához hozzunk létre egy minta dataframe-et, majd alakítsuk át a szín változót dummy változóvá.

In [1]:
import pandas as pd

df_minta=pd.DataFrame({'Nev':['Aranka', 'Piroska', 'Józsi', 'Benedek'],
                        'Szemszin': ['kék', 'zöld', 'barna', 'zöld']})
df_minta

,Nev,Szemszin
0,Aranka,kék
1,Piroska,zöld
2,Józsi,barna
3,Benedek,zöld


In [2]:
# a Szemszin oszlop on-hot-encoding átalakítása:
one_hot = pd.get_dummies(df_minta['Szemszin'])
one_hot

,barna,kék,zöld
0,0,1,0
1,0,0,1
2,1,0,0
3,0,0,1


In [3]:
#hozzáfűzés az eddigi dataframe-hez és a korábbi oszlop eldobása:
df_minta = df_minta.drop('Szemszin',axis = 1)
df_minta = df_minta.join(one_hot)
df_minta

,Nev,barna,kék,zöld
0,Aranka,0,1,0
1,Piroska,0,0,1
2,Józsi,1,0,0
3,Benedek,0,0,1


## Adatmigráció

Első lépésként olvassuk be a *fiktiv_szemely1.csv* és a *fiktiv_szemely2.csv* fájlokat, majd migráljuk őket!

In [4]:
import numpy as np
import pandas as pd

szem1 = pd.read_csv('C:\\Dropbox\\0_DSalapjai\\Python_programok\\fiktiv_szemely1.csv',
                    sep=';', header=0)
szem2 = pd.read_csv('C:\\Dropbox\\0_DSalapjai\\Python_programok\\fiktiv_szemely2.csv',
                    sep=';', header=0)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Dropbox\\0_DSalapjai\\Python_programok\\fiktiv_szemely1.csv'

Kukkantsunk bele, hogy milyen adatokat tartalmaznak.

In [ ]:
szem1.head(5)

In [ ]:
szem2.head(5)

Migráljuk a 2 állományt!

In [ ]:
szem = pd.concat([szem1, szem2], ignore_index=True)

In [ ]:
szem.head(5)

Ha csak a DataFrame oszlopneveinet  szeretnénk lekérni, akkor ezt a **list** függvénnyel valósíthatjuk meg.

In [ ]:
list(szem)

Mekkora a migrált adathalmaz mérete?

In [ ]:
szem.shape

In [ ]:
szem1.shape

In [ ]:
szem2.shape

A felesleges változókat pedig töröljük!

In [ ]:
del szem1
del szem2

## Adattisztítás

### Hibás adatok kezelése

Vizsgáljuk meg közelebbről a numerikus értékeket tartalmazó oszlopok adatait!  
A numerikus értékek legnagyobb és legkisebb értékeit az **nlargest** és **nsmallest** függvényekkel jeleníthetjük meg. 

In [ ]:
szem.nlargest(10, 'eletkor')

In [ ]:
szem.nsmallest(10, 'eletkor')

A nyilvánvaló adathibák esetében egyik lehetséges (és egyben legegyszerűbb) mód a hibás adatok törlése (nem túl sok hibás adat esetén ajánlott csak).   
Töröljük azon életkor adatokat, ahol az életkor nem a [18, 120] intervallumba eseik!

In [ ]:
szem.loc[szem.eletkor > 120, 'eletkor'] = np.nan
szem.loc[szem.eletkor < 18, 'eletkor'] = np.nan

Csupán ellenőrzésképpen nézzük meg, hogy pl a 20-30-as idexű rekordok adatai hogyan módosultak. 

In [ ]:
szem[20:30]

Hasonlóan járjunk el a *testsuly* és *testmagassag* adatok esetében is!

In [ ]:
szem.nlargest(10, 'testsuly')

In [ ]:
szem.nsmallest(10, 'testsuly')

In [ ]:
szem.loc[szem.testsuly > 200, 'testsuly'] = np.nan
szem.loc[szem.testsuly < 40, 'testsuly'] = np.nan

In [ ]:
szem.nlargest(10, 'testmagassag')

In [ ]:
szem.nsmallest(10, 'testmagassag')

In [ ]:
szem.loc[szem.testmagassag > 250, 'testmagassag'] = np.nan
szem.loc[szem.testmagassag < 100, 'testmagassag'] = np.nan

### Hiányzó adatok és duplumok kezelése

#### Üres adatok lekérdezése

Az **isnull** függvény segítségével le tudjuk kérdezni, hogy a DataFrame-ben hol találunk NAN értékeket:

In [ ]:
szem.isnull()  #vagy: pd.isnull(szem)

Mivel ez így eléggé átláthatatlan, ezért célszerű továbbfűzni az előző gondolatot és lekérdezni azon sorok indexeit, amelyek tartalmaznak NAN értéket.
Ehhez első lépésben nézzük meg, hogy hogyan lehet lekérdezni, hogy mely sorok/oszlopok tartalmazak NAN értéket. Erre sok lehetőségünk van, megvalósíthatjuk például így is:   
</br>   
Mely oszlopok tartalmaznak NAN adatot?

In [ ]:
szem.isnull().any(axis=0)  

Mely sorok tartalmaznak NAN adatot?

In [ ]:
szem.isnull().any(axis=1)

Az üres adatot tartalmazó sorok indexeit többféleképpen is kilistázhatjuk. Leggyorsabb talán a *szem.isnull().any(axis=1).nonzero()* utasítás, de hasonló eredményt érhetünk el a következőképpen is:

In [ ]:
szem[szem.isnull().any(1)==True].index

Az előző utasítások int számokat adott vissza, ha szeretnénk könnyen kezelni őket, akkor listává érdemes konvertálni, vagy egy tömbbe tenni őket:

In [ ]:
szem[szem.isnull().any(1)==True].index.tolist()

In [ ]:
np.array(szem[szem.isnull().any(1)==True].index)

Ha arra vagyunk kiváncsiak, hogy hány olyan rekordunk van, amely NaN adatot tartalmaz, akkor az előző lista, vagy tömb méretét kell lekérdezni. De hasonló eredményt kapnánk a *szem.isnull().any(1).nonzero()[0].size* utasítással is.

In [ ]:
len(szem[szem.isnull().any(1)==True].index.tolist())

In [ ]:
np.array(szem[szem.isnull().any(1)==True].index).size

#### Hiányzó adatok pótlása

**Manuális pótlás**: Ha valamely adatot utólagosan beszereztük, akkor könnyen tudjuk  a DataFrame-ben is pótolni.   
Tegyük fel, hogy a *19*-es *id*-jú személy testmagasságát utólagosan megtudtuk, s ez *180cm*.

In [ ]:
szem.loc[szem.id == 19]

In [ ]:
szem.loc[szem.id == 19, 'testmagassag'] = 180
szem.loc[szem.id == 19]

**Feltöltés globális konstanssal**: ebben a műveletben a **replace** függvény lesz segítségünkre, a hiányzó értékekre pedig az **np.nan** függvénnyel hivatkozhatunk.  
Cseréljül le a hiányzó *eletkor* adatokat *ismeretlen* értékekre.   
Először vizsgáljuk meg, hogy hol tartalmaz NaN adatokat az *eletkor* mező.

In [ ]:
szem[szem['eletkor'].isnull()].index.tolist()

In [ ]:
szem.iloc[151]

In [ ]:
szem.eletkor.replace(np.nan, 'ismeretlen', inplace=True)
szem.iloc[151]

Mivel a NAN értékeket sokkal könnyebben tudjuk kezelni, ezért írjuk vissza az *eletkor* oszlopba a *NAN* adatokat az *'ismeretlen'* adatok helyett.

In [ ]:
szem.eletkor.replace('ismeretlen', np.nan, inplace=True)
szem.iloc[151]

#### Hiányzó értékeket tartalmazó rekordok törlése

A hiányzó adatok törlése a **dropna** metódussal valósítható meg. Segítségével törölhetünk:  
- sorokat (*axis=0*),
- oszlopokat (*axis=1*) 

valamit olyan sorokat/oszlopokat, amelyekben:
- legalább 1 NAN adat van (*how='any'*)
- minden adat NAN (*how='all'*).

In [ ]:
szem.dropna(axis=0, how='all', inplace=True)
szem.shape

In [ ]:
szem.dropna(axis=0, how='any', inplace=True)
szem.shape

## Adattranszformáció

Hozzunk létre egy új oszlopot *bmi* néven, amely az egyes személykre kiszámított testtömeg indexet tartalmazza. A testömeg index a kilogrammban mért testtömeg és a méterben mért tesmagasság négyzetének hányadosa.

### Származtatott oszlop

In [ ]:
szem['bmi'] = szem['testsuly']/((szem['testmagassag']/100)**2)
szem.head(20)

### Adatdiszkretizáció

Az adatok diszretizálásának egyik lehetséges módja a **vödrözési technika** alkalmazása.  
Egyenlőszélességi vödröket a **cut** metódussal, egyenlő mélységű vödröket pedig a **qcut** metódussal  tudunk  kialakítani.
<br>  
Hozzunk létre *korcsopnev* néven új attribútumot, ahova a személyeket diszkretizáljuk életkoruk alapján *fiatal*, *középkorú* és *idős*  kategóriákba **egyenlő szélességű vödrök** alkalmazásával!

In [ ]:
szem['korcsopnev'] = pd.cut(szem.eletkor, 3)
szem.head(10)

In [ ]:
szem['korcsopnev'] = pd.cut(szem.eletkor, 3, 
                            labels=['fiatal', 'kozepkoru', 'idos'])
szem.head(10)

Tegyük ugyanezt meg **egyenlő mélységű vödrök** alkalmazásával, s az eredmény tároljuk el a *korcsopnev2* oszlopban.  

In [ ]:
szem['korcsopnev2'] = pd.qcut(szem.eletkor, 3, 
                              labels=['fiatal', 'kozepkoru', 'idos'])
szem.head(10)

Mivel a későbbiekben a *korcsopnev2* oszlopra nincs szükségünk, azért töröljük:

In [ ]:
szem.drop('korcsopnev2', axis=1, inplace=True)
szem.head(10)

## Normalizáció

**Min-max normalizáció**: Normáljuk a *szem* adathalmaz numerikus értékeit a [0, 1] intervallumba a min-max normalizációs módszer alapján! 

In [ ]:
cols = ['testsuly', 'testmagassag', 'eletkor']

In [ ]:
for col in cols:
    col_minmax = col + '_minmaxnorm'
    szem[col_minmax] = (szem[col] - min(szem[col]))/(max(szem[col])-min(szem[col]))
szem.head(10)

Hasonló eredményt értünk volna el a következő utasítás alkalmazásával is:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; *szem[col_minmax] = (szem[col] - szem[col].min())/(szem[col].max()-szem[col].min())*

<br>
**Zéruspont normalizáció**: Normalizáljuk szintén a numerikus adatokat zéruspont normalizációval!

In [ ]:
for col in cols:
    col_zscore = col + '_zscore'
    szem[col_zscore] = (szem[col] - np.mean(szem[col]))/np.std(szem[col])
szem.head(10)

Tételezzük fel, hogy a továbbiakban nincs szükségünk a z-score normált értékekre, azért töröljük ezen oszlopokat!

In [ ]:
szem.drop(['testsuly_zscore', 'testmagassag_zscore', 'eletkor_zscore'], 
          axis=1, inplace=True)
szem.head(10)

### Duplikált sorok törlése

A duplikált sorokat a ** drop_duplicates** metódussal törölhetjük:

In [ ]:
szem.drop_duplicates(inplace=True)
szem.shape

## Mintavételezés

**Visszatevés nélküli mintát** a **sample** metódussal tudunk kiválasztani. Lehetőségünk van egy meghatározott elemszámú mintát, illetve a teljes adathalmaz valahány százalékát is kiválasztani. 

In [ ]:
szem.sample(10)

In [ ]:
szem.sample(frac=0.02)

**Visszatevéses mintavételezés**: az **iloc** és az **np.random.choice** függvények segítségével valósítható meg:

In [ ]:
szem.iloc[np.random.choice(szem.index, 10)]

## Tisztított adatok mentése

Ha már ennyit dolgoztunk vele, akkor a tisztított migrált adathalmazt írjuk ki egy csv fájlba.

In [ ]:
szem.to_csv('C:\\Dropbox\\0_DSalapjai\\Python_programok\\fiktiv_szemely_migralt.csv',
            sep=';', header=True, index=False)